In [1]:
%matplotlib inline
import matplotlib.pylab as plt

In [17]:
import numpy as np

In [2]:
from pycrates import read_file

In [3]:
from coords.utils import point_separation

In [4]:
tab = read_file("obspar.fits")

In [8]:
print(tab.get_colnames())

['parameter_file_name', 'title', 'observer', 'ao', 'object', 'ss_object', 'obs_id', 'obi_num', 'seq_num', 'instrume', 'grating', 'detector', 'detnam', 'si_mode', 'optical_monitor', 'raster_scan', 'dither', 'phase_constraint', 'ra_pnt', 'dec_pnt', 'roll_pnt', 'roll_constraint', 'roll_angle', 'roll_tol', 'roll_180_allowed', 'ra_targ', 'dec_targ', 'y_det_offset', 'z_det_offset', 'radial_offset', 'window_constraint', 'window_start', 'window_stop', 'defocus', 'sim_z_offset', 'pre_min_lead', 'pre_max_lead', 'pre_id', 'uninterrupted', 'seg_max_num', 'ra_nom', 'dec_nom', 'roll_nom', 'date-obs', 'date-end', 'tstart', 'tstop', 'sched_start', 'sched_stop', 'sched_exp_time', 'obs_mode', 'maneuver', 'maneuver_v1', 'maneuver_v2', 'maneuver_v3', 'maneuver_angle', 'maneuver_ref', 'mjdref', 'timezero', 'timeunit', 'timesys', 'timversn', 'datamode', 'trigger_level', 'range_switch_level', 'spect_mode', 'antico_enable', 'width_enable', 'width_threshold', 'uld_enable', 'upper_level_disc', 'blank_enable', '

In [42]:
obsid = tab.obs_id.values
obi   = tab.obi_num.values
inst  = tab.instrume.values
det   = tab.detnam.values
obsmode = tab.obs_mode.values
dmode = tab.datamode.values
rmode = tab.readmode.values
ra = tab.ra_pnt.values
dec = tab.dec_pnt.values

In [21]:
print(set(rmode[inst=='ACIS']))
print(set(dmode[inst=='ACIS']))
print(set(obsmode))

{'TIMED', 'CONTINUOUS'}
{'FAINT_BIAS', 'VFAINT', 'FAINT', 'RAW', 'CC33_FAINT', 'CC33_GRADED', 'GRADED'}
{'POINTING', 'SECONDARY'}


In [27]:
acis_only = np.where(inst=="ACIS")
te_only = np.where(rmode=="TIMED")
pointing = np.where(obsmode=="POINTING")

In [39]:
myobs, = np.where( (inst=='ACIS') & (rmode=='TIMED') & (obsmode=='POINTING'))

In [44]:
from collections import namedtuple
Obs = namedtuple( 'Obs', ['obsid','obinum', 'ra', 'dec'])

In [49]:
mylist = [Obs(obsid[i],obi[i],ra[i],dec[i]) for i in myobs]

In [54]:
SEPARATION_THRESHOLD = 20/3600.0 # arcsec diff in pointing

In [66]:
retval = {}
n_obs = len(mylist)
for ii in range(n_obs-1):    

    if (ii%500)==0:
        print(ii)

    obsA = mylist[ii]
    
    ra_a = obsA.ra
    dec_a = obsA.dec
    
    for jj in range(ii+1,n_obs):
        obsB = mylist[jj]
        ra_b = obsB.ra
        dec_b = obsB.dec

        if np.abs(dec_a-dec_b) > 3*SEPARATION_THRESHOLD:
            continue        
        
        d = point_separation(ra_a, dec_a, ra_b,dec_b)
        if d <= SEPARATION_THRESHOLD:
            if obsA in retval:
                retval[obsA].append(obsB)
            else:
                retval[obsA]=[obsB]
            
            if obsB in retval:
                retval[obsB].append(obsA)
            else:
                retval[obsB]=[obsA]
            

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000


In [67]:
retval

{Obs(obsid='11512', obinum=0, ra=187.70193948480139, dec=12.38727657891058): [Obs(obsid='8580', obinum=0, ra=187.70445815899151, dec=12.386929547391119),
  Obs(obsid='20035', obinum=0, ra=187.7058623308088, dec=12.38694375572371),
  Obs(obsid='18838', obinum=0, ra=187.70629601239111, dec=12.389948778757841),
  Obs(obsid='10286', obinum=0, ra=187.70392839668941, dec=12.38679208490804),
  Obs(obsid='18837', obinum=0, ra=187.70564638147479, dec=12.389325081872061),
  Obs(obsid='16043', obinum=0, ra=187.703071826317, dec=12.38873775723231),
  Obs(obsid='20034', obinum=0, ra=187.7049468795966, dec=12.385384283050611),
  Obs(obsid='18233', obinum=0, ra=187.70142219031439, dec=12.39212313659063),
  Obs(obsid='18783', obinum=1, ra=187.7056219777115, dec=12.38933454194302),
  Obs(obsid='11513', obinum=0, ra=187.7020238569701, dec=12.38727528120255),
  Obs(obsid='8579', obinum=0, ra=187.70376583590971, dec=12.38690194225485),
  Obs(obsid='7351', obinum=0, ra=187.69936241652471, dec=12.3903770370

In [68]:
len(retval)

5972

In [69]:
len(myobs)

15055

In [72]:
ll = [len(retval[x]) for x in retval]

In [86]:
def myid(oo):
    rr = "o{}_{}".format(oo.obsid,oo.obinum)
    return(rr)
   

In [87]:
with open("match.dat","w") as fp:
    fp.write( "digraph chandra { \n" )
    
    
    for k in retval:
        mm = myid(k)
        ll = ",".join([myid(x) for x in retval[k]])
        fp.write("{} -> {{ {} }};\n".format(mm,ll))
    
    fp.write("}\n")

In [96]:
%%bash 
neato  -Tpng match.dat  > foo.png

In [101]:
#from IPython.display import Image
#Image(filename='foo.png') 
